<a href="https://colab.research.google.com/github/RiccardoRubini93/SWA_DQ/blob/main/Check_File_Structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Authentication in GCP with current user credentials

In [5]:
from google.colab import auth

auth.authenticate_user()

print('Authenticated')

Authenticated


Get GCS object info

In [6]:
from io import BytesIO, StringIO
from google.cloud import storage

def get_byte_fileobj(project: str,
                     bucket: str,
                     path: str) -> BytesIO:
    """
    Retrieve data from a given blob on Google Storage and pass it as a file object.
    :param path: path within the bucket
    :param project: name of the project
    :param bucket_name: name of the bucket
    :return: file object (BytesIO)
    """
    blob = _get_blob(bucket, path, project)
    byte_stream = BytesIO()
    blob.download_to_file(byte_stream)
    byte_stream.seek(0)
    return byte_stream

def get_bytestring(project: str,
                   bucket: str,
                   path: str) -> bytes:
    """
    Retrieve data from a given blob on Google Storage and pass it as a byte-string.
    :param path: path within the bucket
    :param project: name of the project
    :param bucket_name: name of the bucket
    :return: byte-string (needs to be decoded)
    """
    blob = _get_blob(bucket, path, project)
    s = blob.download_as_string()
    return s


def _get_blob(bucket_name, path, project):

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(path)
    return blob

In [7]:
import pandas as pd
import gcsfs
import traceback
from google.cloud import storage

########### This version prints both RIGHTLY FORMATTED THEN ERRORS

def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""
    # bucket_name = "your-bucket-name" --> provided as input of the function
    project = "advanced-analytics-278408"
    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name)

    # Note: The call returns a response only when the iterator is consumed.
    # https://gcsfs.readthedocs.io/en/latest/#credentials
    for blob in blobs:
        file_name = "gs://" + bucket_name +  blob.name
        path = blob.name
        if file_name.endswith('.csv'):
          print("############################################## File under analysis: " + file_name)
          print("")
          #df = dd.read_csv(file_name)
          fileobj = get_byte_fileobj(project, bucket_name, path)
          try:
            df = pd.read_csv(fileobj, sep ='\t' )
            print(df.info())
          except Exception as e:
            print("")
            print("##############  ERROR ##############")
            print("")
            print(e)
            print("")
            print("##############  ERROR ##############")
            print("")

In [9]:
list_blobs_errors("data_exchange__crm_prod")

1 out of total read files 5 - ############################################## File under analysis: gs://data_exchange__crm_prodraw/02_SAP_P60/0201/v1/2023/0201__crm_master_MAIN__dd__20230117/0201__crm_master_MAIN__dd__20230117__15360517.csv

##############  ERROR ##############

Error tokenizing data. C error: EOF inside string starting at row 301
301
0000242502	Mumbai	IN		false	400018		"Poonam Chambers, ""G"" Block, Dr.					false						R3BP	X	ZZ	TOUR	AM	Govan Travels				

##############  ERROR ##############

2 out of total read files 9 - ############################################## File under analysis: gs://data_exchange__crm_prodraw/02_SAP_P60/0201/v1/2023/0201__crm_master_MAIN__dd__20230117/0201__crm_master_MAIN__dd__20230117__153621666.csv

##############  ERROR ##############

Error tokenizing data. C error: EOF inside string starting at row 815
815
0000242871	Woronesch	RU		false	394000		"Pr. Revolutzie , "" ZUM"" Worone					false						R3BP	X	ZZ	TOUR	AM	L' Ark				

##############

KeyboardInterrupt: ignored

In [8]:
pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import gcsfs
import traceback
from google.cloud import storage

########### This version prints both RIGHTLY FORMATTED THEN ERRORS

def list_blobs_errors(bucket_name):
    """Lists all the blobs in the bucket."""
    # bucket_name = "your-bucket-name" --> provided as input of the function
    project = "advanced-analytics-278408"
    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name)

    # Note: The call returns a response only when the iterator is consumed.
    # https://gcsfs.readthedocs.io/en/latest/#credentials
    error_count = 0
    total_read_files = 0
    for blob in blobs:
        total_read_files = total_read_files +1
        file_name = "gs://" + bucket_name +  blob.name
        path = blob.name
        if file_name.endswith('.csv'):
          #print("############################################## File under analysis: " + file_name)
          #print("")
          #df = dd.read_csv(file_name)
          fileobj = get_byte_fileobj(project, bucket_name, path)
          try:
            df = pd.read_csv(fileobj, sep ='\t')
            #print(df.info())
          except Exception as e:
            error_count = error_count + 1
            print(str(error_count) + " out of total read files " + str(total_read_files) + " - ############################################## File under analysis: " + file_name)
            print("")
            print("##############  ERROR ##############")
            print("")
            print(e)
            error_string = str(e)
            if "Error tokenizing data. C error: EOF inside string starting at row " in error_string:
              error_line = error_string.replace("Error tokenizing data. C error: EOF inside string starting at row ","")
              print(error_line)
              line = int(error_line)
              with blob.open("r") as f:
                #print(f.read())
                file_text=f.read()
                error_line = file_text.split('\n')[line]
                print(error_line)
            print("")
            print("##############  ERROR ##############")
            print("")

In [ ]:
def test_single_file(project, bucket_name, path):
          fileobj = get_byte_fileobj(project, bucket_name, path)
          blob = _get_blob(bucket_name, path, project)
          print("############################################## File under analysis: gs://" + bucket_name + path)
          try:
            df = pd.read_csv(fileobj, sep ='\t' )
            #print(df.info())
          except Exception as e:
            print("")
            print("##############  ERROR ##############")
            print("")
            print(e)
            error_string = str(e)
            if "Error tokenizing data. C error: EOF inside string starting at row " in error_string:
              error_line = error_string.replace("Error tokenizing data. C error: EOF inside string starting at row ","")
              print(error_line)
              line = int(error_line)
              with blob.open("r") as f:
                #print(f.read())
                file_text=f.read()
                error_line = file_text.split('\n')[line]
                print(error_line)
            print("")
            print("##############  ERROR ##############")
            print("")

In [12]:
def test_single_file_tokenization(project, bucket_name, path):
          fileobj = get_byte_fileobj(project, bucket_name, path)
          blob = _get_blob(bucket_name, path, project)
          print("############################################## File under analysis: gs://" + bucket_name + path)
          #try:
            #df = pd.read_csv(fileobj, sep ='\t' )
            #print(df.info())
          #except Exception as e:
           # print("")
           # print("##############  ERROR ##############")
           # print("")
           # print(e)
            #error_string = str(e)
          with blob.open("r") as f:
               #print(f.read())
               file_text=f.read()
               all_lines_tokenized = file_text.split('\n')
               print("Total lines: " + str(len(all_lines_tokenized)))
               first_line_tokenized = all_lines_tokenized[0].split("\t")
               header_token_count = len(first_line_tokenized)
               print("Fields in the first line: "+ str(len(first_line_tokenized)))
               field_order = 0
               for field in first_line_tokenized:
                 field_order = field_order+1
                 print(str(field_order) + "\t" + field)
               line_count = 0
               for line in all_lines_tokenized:
                 field_order_current = 0
                 line_count = line_count = line_count +1
                 current_line_tokenized = line.split("\t")
                 if len(current_line_tokenized) != header_token_count:
                   print("Line " + str(line_count) + " --> has a different number of fields (expected "+str(header_token_count)+") found: " + str(len(line.split("\t"))))
                   for field in current_line_tokenized:
                     field_order_current = field_order_current+1
                     print(str(field_order) + "\t" + field)
          print("")
          print("##############  ERROR ##############")
          print("")

In [13]:
project = "advanced-analytics-278408"
bucket_name = "data_exchange__crm_prod"
path= "raw/02_SAP_P60/0201/v1/2023/0201__crm_master_MAIN__dd__20230117/0201__crm_master_MAIN__dd__20230117__153835912.csv"
test_single_file_tokenization(project, bucket_name, path)



############################################## File under analysis: gs://data_exchange__crm_prodraw/02_SAP_P60/0201/v1/2023/0201__crm_master_MAIN__dd__20230117/0201__crm_master_MAIN__dd__20230117__153835912.csv
Total lines: 1002
Fields in the first line: 28
1	idnum
2	address_city
3	address_cntry
4	address_district
5	address_inval
6	address_pstcd
7	address_state
8	address_stret
9	address_stret2
10	address_stret3
11	communication_email
12	phone_number
13	person_delet
14	person_bdate
15	person_corla
16	person_origin
17	person_sex
18	mail_domain
19	bu_group
20	classification_is_customer
21	classification_industry
22	classification_account_group
23	classification_kukla
24	name_org1
25	name_org2
26	klabc
27	local_cluster_id
28	local_cluster_name
Line 1002 --> has a different number of fields (expected 28) found: 1
28	

##############  ERROR ##############



In [34]:
def check_for_string_in_blob(project, bucket_name, path,tgt_string):
          fileobj = get_byte_fileobj(project, bucket_name, path)
          blob = _get_blob(bucket_name, path, project)
          with blob.open("r") as f:
               #print(f.read())
               file_text=f.read()
          if tgt_string in file_text:
            #print(f'String found in {path}')
            return file_text.count(tgt_string)
          else:
            return 0
          

          

In [35]:
#get all blob in a bucket
project_id = 'advanced-analytics-278408'
client = storage.Client(project_id)

bucket = 'data_exchange__crm_prod'

count = 0
file_cnt  = 0


for blob in client.list_blobs(bucket, prefix='raw/02_SAP_P60/0201/v1/2023/0201__crm_master_MAIN__dd__20230117'):
  #print(blob.name)
  file_cnt += 1
  #print(f"File count {file_cnt}")
  path = blob.name
  tgt_string = 'ZCNS'
  out = check_for_string_in_blob(project_id,bucket,path,tgt_string)
  
  if out > 0 : 
    count += 1
    print(f"{count} on {file_cnt} files have the target string in the text. Occurencies {out}. Blob name : {blob.name}")


1 on 3 files have the target string in the text. Occurencies 709. Blob name : raw/02_SAP_P60/0201/v1/2023/0201__crm_master_MAIN__dd__20230117/0201__crm_master_MAIN__dd__20230117__153606736.csv
2 on 132 files have the target string in the text. Occurencies 277. Blob name : raw/02_SAP_P60/0201/v1/2023/0201__crm_master_MAIN__dd__20230117/0201__crm_master_MAIN__dd__20230117__154357660.csv
3 on 136 files have the target string in the text. Occurencies 85. Blob name : raw/02_SAP_P60/0201/v1/2023/0201__crm_master_MAIN__dd__20230117/0201__crm_master_MAIN__dd__20230117__161551941.csv
4 on 137 files have the target string in the text. Occurencies 82. Blob name : raw/02_SAP_P60/0201/v1/2023/0201__crm_master_MAIN__dd__20230117/0201__crm_master_MAIN__dd__20230117__16155211.csv
5 on 138 files have the target string in the text. Occurencies 81. Blob name : raw/02_SAP_P60/0201/v1/2023/0201__crm_master_MAIN__dd__20230117/0201__crm_master_MAIN__dd__20230117__161552455.csv
6 on 139 files have the target 